In [2]:
import sqlite3

### CREATE

In [3]:
from pathlib import Path
import pandas as pd

# Use sqlite3 to connect to tips.db
con=sqlite3.connect('tips.db')
# Define cursor to execute SQL commmand
cur=con.cursor()


Show All tables in the tips.db

In [4]:
# Read csv to pandas dataframe
tips=pd.read_csv('data/tips.csv')
# transform pandas dataframe to Database Table
tips.to_sql('TIPS',con,if_exists='replace', index=False)

244

In [5]:
cur.execute(''' SELECT name FROM sqlite_master WHERE type='table' ''')
alltables=cur.fetchall()

for table in alltables:
    print(table[0])


TIPS


SHOW Content in Table Tips

In [6]:
cur.execute(''' SELECT * FROM TIPS''')
print(cur.fetchall())

[(16.99, 1.01, 'Female', 'No', 'Sun', 'Dinner', 2), (10.34, 1.66, 'Male', 'No', 'Sun', 'Dinner', 3), (21.01, 3.5, 'Male', 'No', 'Sun', 'Dinner', 3), (23.68, 3.31, 'Male', 'No', 'Sun', 'Dinner', 2), (24.59, 3.61, 'Female', 'No', 'Sun', 'Dinner', 4), (25.29, 4.71, 'Male', 'No', 'Sun', 'Dinner', 4), (8.77, 2.0, 'Male', 'No', 'Sun', 'Dinner', 2), (26.88, 3.12, 'Male', 'No', 'Sun', 'Dinner', 4), (15.04, 1.96, 'Male', 'No', 'Sun', 'Dinner', 2), (14.78, 3.23, 'Male', 'No', 'Sun', 'Dinner', 2), (10.27, 1.71, 'Male', 'No', 'Sun', 'Dinner', 2), (35.26, 5.0, 'Female', 'No', 'Sun', 'Dinner', 4), (15.42, 1.57, 'Male', 'No', 'Sun', 'Dinner', 2), (18.43, 3.0, 'Male', 'No', 'Sun', 'Dinner', 4), (14.83, 3.02, 'Female', 'No', 'Sun', 'Dinner', 2), (21.58, 3.92, 'Male', 'No', 'Sun', 'Dinner', 2), (10.33, 1.67, 'Female', 'No', 'Sun', 'Dinner', 3), (16.29, 3.71, 'Male', 'No', 'Sun', 'Dinner', 3), (16.97, 3.5, 'Female', 'No', 'Sun', 'Dinner', 3), (20.65, 3.35, 'Male', 'No', 'Sat', 'Dinner', 3), (17.92, 4.08,

### READ

1. Retrieve the average tip percentage for each day of the week

In [7]:
cur.execute('''
    SELECT DAY,SUM(tip)/SUM(total_bill)*100 AS avg_tip_percentage 
    FROM TIPS
    GROUP BY DAY
''')

print(cur.fetchall())

[('Fri', 15.94451945501412), ('Sat', 14.642375168690961), ('Sun', 15.20379065365422), ('Thur', 15.67320058741438)]


2.Find the maximum and minimum total bull amounts

In [8]:
cur.execute('''
SELECT MAX(total_bill) AS max_total_bill,MIN(total_bill) AS min_total_bill 
FROM TIPS
''')

print(cur.fetchall())

[(50.81, 3.07)]


3.Count the number of parties for each size

In [9]:
cur.execute('''
    SELECT size,COUNT(*) AS number 
    FROM TIPS
    GROUP BY size
''')

print(cur.fetchall())

[(1, 4), (2, 156), (3, 38), (4, 37), (5, 5), (6, 4)]


4.Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%

In [10]:
cur.execute('''
    SELECT total_bill,tip 
    FROM TIPS
    WHERE size>=4 AND tip/total_bill>0.15
''')

print(cur.fetchall())

[(25.29, 4.71), (18.43, 3.0), (39.42, 7.58), (30.4, 5.6), (32.4, 6.0), (25.56, 4.34), (18.29, 3.76), (29.93, 5.07), (34.3, 6.7), (27.05, 5.0), (29.85, 5.14), (21.5, 3.5), (23.17, 6.5), (20.69, 5.0), (20.53, 4.0), (25.89, 5.16), (48.33, 9.0)]


5.Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order

In [11]:
cur.execute('''
    SELECT day,time,SUM(total_bill) AS sum_total_bill,SUM(tip) AS tip_amount,SUM(tip)/SUM(total_bill)*100 AS tip_percentage
    FROM TIPS
    GROUP BY day,time
    ORDER BY tip_percentage DESC
''')

print(cur.fetchall())

[('Fri', 'Lunch', 89.92, 16.68, 18.54982206405694), ('Thur', 'Dinner', 18.78, 3.0, 15.974440894568689), ('Thur', 'Lunch', 1077.5499999999997, 168.83, 15.667950443134895), ('Sun', 'Dinner', 1627.1600000000003, 247.39000000000007, 15.20379065365422), ('Fri', 'Dinner', 235.95999999999998, 35.28, 14.951686726563826), ('Sat', 'Dinner', 1778.3999999999996, 260.4, 14.642375168690961)]


6.Find the average tip percentage for each combination of day, time, and smoker status

In [12]:
cur.execute('''
    SELECT day,time,smoker,SUM(tip)/SUM(total_bill)*100 AS tip_percentage
    FROM TIPS
    GROUP BY day,time,smoker
''')

print(cur.fetchall())

[('Fri', 'Dinner', 'No', 14.298093587521665), ('Fri', 'Dinner', 'Yes', 15.16324469875463), ('Fri', 'Lunch', 'No', 18.773466833541928), ('Fri', 'Lunch', 'Yes', 18.50148769272383), ('Sat', 'Dinner', 'No', 15.781324170980351), ('Sat', 'Dinner', 'Yes', 13.514693046261279), ('Sun', 'Dinner', 'No', 15.448121278488811), ('Sun', 'Dinner', 'Yes', 14.580605743213754), ('Thur', 'Dinner', 'No', 15.974440894568689), ('Thur', 'Lunch', 'No', 15.615391782353496), ('Thur', 'Lunch', 'Yes', 15.788989700833739)]


7.Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records

In [17]:
query7 = '''SELECT sex, SUM(total_bill), SUM(tip), (SUM(tip) / SUM(total_bill)) * 100 AS tip_percentage 
    FROM TIPS
    GROUP BY sex
    ORDER BY total_bill DESC 
    LIMIT 5;'''
df7 = pd.read_sql_query(query7, con)
print(df7)

      sex  SUM(total_bill)  SUM(tip)  tip_percentage
0  Female          1570.95    246.51       15.691779
1    Male          3256.82    485.07       14.893976


8.Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount

In [33]:
query8 = '''
SELECT *
FROM(
SELECT SUM(tip) as sum_tips, SUM(total_bill) as sum_total_bill, SUM(tip)/SUM(total_bill)*100 AS percentage
FROM TIPS
GROUP BY day,time)
WHERE percentage >= (
SELECT MAX(percentage)
FROM(
SELECT SUM(tip) as sum_tips, SUM(total_bill) as sum_total_bill, SUM(tip)/SUM(total_bill)*100 AS percentage
FROM TIPS
GROUP BY day,time) ) 
OR percentage <= (
SELECT MIN(percentage)
FROM(
SELECT SUM(tip) as sum_tips, SUM(total_bill) as sum_total_bill, SUM(tip)/SUM(total_bill)*100 AS percentage
FROM TIPS
GROUP BY day,time))'''
df8 = pd.read_sql_query(query8, con)
print(df8)

   sum_tips  sum_total_bill  percentage
0     16.68           89.92   18.549822
1    260.40         1778.40   14.642375


9. Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100

In [37]:
query9 = '''
SELECT SUM(total_bill) AS total_bill, 
       SUM(tip) AS total_tip, 
       (SUM(tip) / SUM(total_bill)) * 100 AS tip_percentage
FROM TIPS
WHERE size >= 4
GROUP BY size
HAVING tip_percentage > 15 
   AND SUM(total_bill) BETWEEN 50 AND 100;
'''

df9 = pd.read_sql_query(query9, con)
print(df9)


Empty DataFrame
Columns: [total_bill, total_tip, tip_percentage]
Index: []


10. Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records

In [44]:
query10 = '''
SELECT day, time, smoker, AVG(tip / total_bill * 100) AS avg_tip_percentage 
FROM TIPS 
GROUP BY day, time, smoker
HAVING COUNT(*) > 5;
'''

df10 = pd.read_sql_query(query10, con)
print(df10)

    day    time smoker  avg_tip_percentage
0   Fri  Dinner    Yes           16.534736
1   Fri   Lunch    Yes           18.893659
2   Sat  Dinner     No           15.804766
3   Sat  Dinner    Yes           14.790607
4   Sun  Dinner     No           16.011294
5   Sun  Dinner    Yes           18.725032
6  Thur   Lunch     No           16.031067
7  Thur   Lunch    Yes           16.386327


1st addtional Querry:
Retrieve the tips from descending order

In [45]:
query11 = '''
SELECT total_bill, tip
FROM TIPS
ORDER BY tip DESC
'''

df11 = pd.read_sql_query(query10, con)
print(df11)

    day    time smoker  avg_tip_percentage
0   Fri  Dinner    Yes           16.534736
1   Fri   Lunch    Yes           18.893659
2   Sat  Dinner     No           15.804766
3   Sat  Dinner    Yes           14.790607
4   Sun  Dinner     No           16.011294
5   Sun  Dinner    Yes           18.725032
6  Thur   Lunch     No           16.031067
7  Thur   Lunch    Yes           16.386327


2nd addtional Querry: Find the total number of smokers and non-smokers

In [46]:
query12 = '''
SELECT smoker, COUNT(*) AS total 
FROM TIPS 
GROUP BY smoker;
'''

df12 = pd.read_sql_query(query12, con)
print(df12)

  smoker  total
0     No    151
1    Yes     93


3rd additional querry: Retrieve the total_bill for parties of size greater than 2

In [50]:
query13 = '''
SELECT total_bill, size
FROM TIPS 
WHERE size >= 2;
'''

df13 = pd.read_sql_query(query13, con)
print(df13)

     total_bill  size
0         16.99     2
1         10.34     3
2         21.01     3
3         23.68     2
4         24.59     4
..          ...   ...
235       29.03     3
236       27.18     2
237       22.67     2
238       17.82     2
239       18.78     2

[240 rows x 2 columns]


4th additional querry: Count the number of meals served during lunch and dinner

In [51]:
query14 = '''
SELECT time, COUNT(*) AS meals_served 
FROM TIPS 
GROUP BY time;
'''

df14 = pd.read_sql_query(query14, con)
print(df14)

     time  meals_served
0  Dinner           176
1   Lunch            68


5th additional query: Retrieve total bills for male and female customers on each day

In [53]:
query15 = '''
SELECT day, sex, SUM(total_bill) AS total_bill 
FROM TIPS 
GROUP BY day, sex;
'''

df15 = pd.read_sql_query(query15, con)
print(df15)

    day     sex  total_bill
0   Fri  Female      127.31
1   Fri    Male      198.57
2   Sat  Female      551.05
3   Sat    Male     1227.35
4   Sun  Female      357.70
5   Sun    Male     1269.46
6  Thur  Female      534.89
7  Thur    Male      561.44
